In [1]:
import copy
import fnmatch
import json
import getpass
import os
import pathlib
import datetime
                    
from dask.distributed import LocalCluster, SSHCluster 
from laserfarm import Retiler, DataProcessing, GeotiffWriter, MacroPipeline
from laserfarm.remote_utils import get_wdclient, get_info_remote, list_remote

def last_modified(opts, remote_path):
    info = get_info_remote(get_wdclient(opts), remote_path.as_posix())
    format_ = '%a, %d %b %Y %H:%M:%S GMT'
    return datetime.datetime.strptime(info['modified'], format_)

# Macro-Pipeline AHN3 Workflow - Feature Extraction (All Points)

## Set Run-Specific Input

Fill in the username/password for the SURF dCache. Choose whether you want to i) run all input files, ii) run the only input files listed in `filename`, or iii) run the input that was updated since the last workflow run.

In [2]:
remote_path_root = pathlib.Path('/pnfs/grid.sara.nl/data/projects.nl/eecolidar/01_Escience/')

# dCache path to retiled data 
remote_path_input = remote_path_root / 'ALS/Netherlands/ahn3_current/ahn3_current_TOP10NL_ud20200323_normalized'

# dCache path where to copy the targets
remote_path_output = remote_path_input.with_name(remote_path_input.name.replace('normalized', 'targets_all'))

run = 'from_file' # 'all', 'updated', 'from_file'
filename = 'feature_extraction_all_failed.json'  # if run is 'from_file', set name of file with input file names
assert run in ['all', 'updated', 'from_file']

In [3]:
webdav_login = input('WebDAV username: ')
webdav_password = getpass.getpass('WebDAV password: ')
if run == 'updated':
    last_run = datetime.datetime.strptime(input('Date last run (YYYY-MM-DD): '), '%Y-%m-%d')

WebDAV username:  fnattin4
WebDAV password:  ············


## Check Connection to Remote Storage

In [4]:
wd_opts = {
    'webdav_hostname': 'https://webdav.grid.surfsara.nl:2880',
    'webdav_login': webdav_login,
    'webdav_password': webdav_password,
    'webdav_timeout': 200
}
assert get_wdclient(wd_opts).check(remote_path_root.as_posix())

In [5]:
tiles = [t for t in list_remote(get_wdclient(wd_opts), remote_path_input.as_posix())
         if fnmatch.fnmatch(t, 'tile_*_*.laz')]
print('Found: {} tiles'.format(len(tiles)))
if run == 'updated':
    # determine which tiles have been updated since last run
    tiles = [t for t in tiles if last_modified(wd_opts, remote_path_input/t) > last_run]
elif run == 'from_file':
    with open(filename, 'r') as f:
        tiles_read = json.load(f)
    # check whether all files are available on dCache
    assert all([t in tiles for t in tiles_read]), f'Some of the files in {filename} are not in remote dir'
    tiles = tiles_read
print('Retrieve and extract features for: {} tiles'.format(len(tiles)))

Found: 37457 tiles
Retrieve and extract features for: 4 tiles


## Setup Cluster

Setup Dask cluster used for all the macro-pipeline calculations.

In [6]:
local_tmp = pathlib.Path('/data/local/tmp')

nprocs_per_node = 2  

# start the cluster
scheduler_node = 'node1'

hosts = [f'node{i}' for i in range(1, 11)]
cluster = SSHCluster(hosts=[scheduler_node] + hosts, 
                     connect_options={'known_hosts': None, 
                                      'username': 'ubuntu', 
                                      'client_keys': '/home/ubuntu/.ssh/id_rsa'},
                     worker_options={'nthreads': 1, 
                                     'nprocs': nprocs_per_node,
                                     'memory_limit': 'auto',
                                     'local_directory': local_tmp/'dask-worker-space'}, 
                     scheduler_options={'dashboard_address': '8787'})
cluster

distributed.deploy.ssh - INFO - distributed.scheduler - INFO - -----------------------------------------------
distributed.deploy.ssh - INFO - /usr/local/lib/python3.7/site-packages/pandas/compat/__init__.py:117: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
distributed.deploy.ssh - INFO - warnings.warn(msg)
distributed.deploy.ssh - INFO - distributed.scheduler - INFO - -----------------------------------------------
distributed.deploy.ssh - INFO - distributed.scheduler - INFO - Clear task state
distributed.deploy.ssh - INFO - distributed.scheduler - INFO -   Scheduler at: tcp://145.100.59.123:8786
distributed.deploy.ssh - INFO - distributed.nanny - INFO -         Start Nanny at: 'tcp://145.100.59.59:35933'
distributed.deploy.ssh - INFO - distributed.nanny - INFO -         Start Nanny at: 'tcp://145.100.59.59:44675'
distributed.deploy.ssh - INFO - distributed.nanny - INFO -         S

SSHCluster('tcp://145.100.59.123:8786', workers=20, threads=20, memory=337.31 GB)

## Feature Extraction

We extract features for all points available.

In [7]:
# details of the retiling schema
grid = {
    'min_x': -113107.81,
    'max_x': 398892.19,
    'min_y': 214783.87,
    'max_y': 726783.87,
    'n_tiles_side': 512
}

# target mesh size & list of features
tile_mesh_size = 10.
features = ['density_absolute_mean_normalized_height', 'pulse_penetration_ratio', 'point_density']

# setup input dictionary to configure the feature extraction pipeline
feature_extraction_input_all = {
    #######
    'log_config': {'level': 'DEBUG'},
    #######
    'setup_local_fs': {'tmp_folder': local_tmp.as_posix()},
    'pullremote': remote_path_input.as_posix(),
    'load': {'attributes': ['raw_classification', 'normalized_height']},
    'generate_targets': {
        'tile_mesh_size' : tile_mesh_size,
        'validate' : True,
        **grid
    },
    'extract_features': {
        'feature_names': features,
        'volume_type': 'cell',
        'volume_size': tile_mesh_size
    },
    'export_targets': {
        'attributes': features,
        'multi_band_files': False
    },
    'clear_cache' : {},
    'pushremote': remote_path_output.as_posix(),
    'cleanlocalfs': {}
}

# write input dictionary to JSON file
with open('feature_extraction_all.json', 'w') as f:
    json.dump(feature_extraction_input_all, f)

In [8]:
macro = MacroPipeline()

# extract the tile indices from the tile names
tile_indices = [[int(el) for el in tile.split('.')[0].split('_')[1:]] for tile in tiles]

# add pipeline list to macro-pipeline object and set the corresponding labels
macro.tasks = [DataProcessing(t, tile_index=idx).config(feature_extraction_input_all).setup_webdav_client(wd_opts) 
               for t, idx in zip(tiles, tile_indices)]
macro.set_labels([os.path.splitext(tile)[0] for tile in tiles])

macro.setup_cluster(cluster=cluster)

# run!
macro.run()

# save outcome results and write name of failed pipelines to file
macro.print_outcome(to_file='feature_extraction_all.out')
failed = macro.get_failed_pipelines()
if failed:
    with open('feature_extraction_all_failed.json', 'w') as f:
        json.dump(['.'.join([pip.label, 'laz']) for pip in failed], f)
    raise RuntimeError('Some of the pipelines have failed')

## Terminate cluster

In [9]:
macro.shutdown()